In [ ]:
import sys
sys.path.append("src")

from build_cnn_model import build_cnn_model
from data_pipeline_clean import train_generator, val_generator, test_generator, class_labels, class_weights

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import tensorflow as tf

# -------------------------------
# بناء النموذج
# -------------------------------
input_shape = (128, 128, 3)   # حجم الصورة
num_classes = len(class_labels)

model = build_cnn_model(input_shape, num_classes)

# تعديل compile ليكون صريح مع Categorical Cross-Entropy
model.compile(
    optimizer='adam',
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
)

print("CNN model ready ✅")
model.summary()

# -------------------------------
# 2️⃣ Callbacks للتدريب
# -------------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# هنسيف أحسن موديل باسم best_model.h5 (أو .keras في النسخ الجديدة)
checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# -------------------------------
#  تدريب الموديل
# -------------------------------
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=40,
    class_weight=class_weights,
    callbacks=[early_stop, checkpoint]
)

# ============================================================
# الجزء الجديد: عرض النتائج بعد التدريب
# ============================================================

# 1️⃣ رسم منحنيات التعلم (Loss & Accuracy Curves)
# ده بيوريك هل فيه Overfitting ولا لأ
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(15, 5))

# رسم الـ Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# رسم الـ Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# 2️⃣ حساب الدقة والـ Loss على بيانات الـ Test (بيانات لم يرها الموديل من قبل)
print("\nEvaluating on Test Set...")
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Loss: {test_loss:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

# 3️⃣ تقرير مفصل (اختياري لكن مفيد جداً)
# بنعمل reset للـ generator عشان نبدأ التوقع من أول صورة
test_generator.reset()

# عمل توقعات (Predictions)
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1) # تحويل الاحتمالات لأرقام كلاسات
true_classes = test_generator.classes # الكلاسات الحقيقية
class_labels_list = list(test_generator.class_indices.keys())

# طباعة تقرير يوضح الـ Precision و Recall لكل كلاس
print("\nDetailed Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels_list))

# (اختياري) رسم الـ Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels_list, yticklabels=class_labels_list)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()

